In [1]:
Vir = lie_algebras.VirasoroAlgebra(QQ)
M = Vir.verma_module(1/2, 1/2)
d = Vir.basis()
v = M.highest_weight_vector()
m2 = (-4*d[-2] + 3*d[-1]*d[-1])*v
m3 = (-12*d[-2]*d[-1] + 4*d[-1]*d[-1]*d[-1] + 3*d[-3])*v
var('t q')

def L_partition_vector(partition, vector):
    for i in range(len(partition)):
        vector = d[-partition[len(partition) - i - 1]]*vector
    return vector

def my_coefficient(partition, vector):
    return vector.monomial_coefficients().get(tuple(-k for k in partition), 0)

def number_of_ones(partition):
    for i in range(len(partition)):
        if partition[len(partition) - i - 1] != 1:
            return i
    return len(partition)

def my_len(partition):
    return 2*len(partition) - number_of_ones(partition)

def row_to_vector(row, n):
    list_of_partition = ordered_partitions(n)
    return sum(row[i]*L_partition_vector(list_of_partition[i], v) for i in range(len(row)))

def series_t_q(list_of_list_of_partition):
    series = 0
    for i in range(len(list_of_list_of_partition)):
        series_t = 0
        for j in range(len(list_of_list_of_partition[i])):
            series_t = series_t + t^my_len(list_of_list_of_partition[i][j])
        series = series + series_t*q^i
    return series

def partition_ends(partition, tail, is_greater):
    if len(partition) < len(tail) - 1:
        return False
    partition2 = partition[len(partition) - len(tail) + 1: len(partition)]
    if partition2 != tail[1:]:
        return False
    if is_greater:
        if partition == partition2:
            return True
        if partition[len(partition) - len(tail)] > tail[0]:
            return True
    else:
        if partition == partition2:
            return False
        if partition[len(partition) - len(tail)] == tail[0]:
            return True
    return False

def list_of_list_of_partition_ends(list_of_list_of_partition, tail, is_greater):
    return [[partition
             for partition in list_of_partition if partition_ends(partition, tail, is_greater)]
            for list_of_partition in list_of_list_of_partition]

def partition_to_polynomial(partition):
    polynomial = [0]*(sum(partition) - 1)
    for i in range(len(partition) - number_of_ones(partition)):
        polynomial[partition[i] - 2] += 1
    return polynomial

def my_order(partition):
    polynomial = partition_to_polynomial(partition)
    polynomial = [-k for k in polynomial]
    polynomial.reverse()
    return ((my_len(partition), len(partition) - number_of_ones(partition), polynomial)) 

def ordered_partitions(n):
    if n <= 0:
        return [[]]
    list_of_partition = list(Partitions(n))
    list_of_partition.sort(key = my_order, reverse = True)
    return list_of_partition

def is_subpartition(partition, partition2):
    if len(partition) < len(partition2):
        return False
    for i in range(len(partition) - len(partition2) + 1):
        if partition[i: i + len(partition2)] == partition2:
            return True
    return False

def partition_contains_any_of_list(partition, list_of_partition):
    for partition2 in list_of_partition:
        if is_subpartition(partition, partition2):
            return True
    return False

def partition_contains_any_of_Q(partition):
    if partition == []:
        return False
    if partition_contains_any_of_list(
        partition,
        [[2], [1, 1, 1], [3, 1, 1], [3, 3], [4, 3, 1], [4, 4, 1], [5, 4, 1, 1], [6, 5, 3, 1]]):
        return True
    if len(partition) > 2:
        for i in range(len(partition) - 2):
            if partition[i] - partition[i + 2] <= 2:
                return True
    if len(partition) > 3:
        for i in range(len(partition) - 3):
            if (partition[i] - partition[i + 3] <= 3
                or (partition[i] - partition[i + 3] <= 4
                    and partition[i + 1] - partition[i + 2] <= 3)):
                return True
    if len(partition) > 4:
        for i in range(len(partition) - 4):
            if (partition[i] - partition[i + 1] == 1 and partition[i + 1] - partition[i + 2] == 2
                and partition[i + 2] - partition[i + 3] == 2
                and partition[i + 3] - partition[i + 4] == 1):
                return True
    return False

def matrix_UVir_m2_m3(n):
    list_of_partition = ordered_partitions(n)
    list_of_partition2 = ordered_partitions(n - 2)
    list_of_partition3 = ordered_partitions(n - 3)
    matrix_m2 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition2[i], m2))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition2))}, sparse = False)
    matrix_m3 = matrix({
        (i, j): my_coefficient(list_of_partition[j], L_partition_vector(list_of_partition3[i], m3))
        for j in range(len(list_of_partition))
        for i in range(len(list_of_partition3))}, sparse = False)
    matrix_m2 = matrix_m2.stack(matrix_m3)
    matrix_m2 = matrix_m2.rref()
    return matrix_m2

def mW_partition(partition):
    n = sum(partition)
    k = ordered_partitions(n).index(partition)
    A_n = matrix_UVir_m2_m3(n)
    pivots = A_n.pivots()
    try:
        return row_to_vector(A_n.row(pivots.index(k)), n)
    except ValueError:
        return 0

def mW_list_of_partition(list_of_partition):
    return [mW_partition(partition) for partition in list_of_partition]

def pivots_up_to(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [list_of_partition[j] for j in matrix_UVir_m2_m3(i).pivots()])
    return list_of_list_of_partition

def basis_up_to(n):
    list_of_list_of_partition = []
    pivots = pivots_up_to(n)
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition if partition not in pivots[i]])
    return list_of_list_of_partition

def basis_up_tov2(n):
    list_of_list_of_partition = []
    for i in range(n + 1):
        list_of_partition = ordered_partitions(i)
        list_of_list_of_partition.append(
            [partition for partition in list_of_partition
             if not partition_contains_any_of_Q(partition)])
    return list_of_list_of_partition

In [2]:
list_of_partition = [[2], [1, 1, 1], [3, 1, 1], [3, 3], [4, 3, 1], [4, 4, 1], [5, 4, 1, 1],
                     [6, 5, 3, 1]]
%time mW_list_of_partition(list_of_partition)

CPU times: user 1min 37s, sys: 94.5 ms, total: 1min 37s
Wall time: 1min 37s


[d[-2]*v - 3/4*d[-1]*d[-1]*v,
 -3*d[-3]*v + d[-1]*d[-1]*d[-1]*v,
 -3/2*d[-5]*v - d[-4]*d[-1]*v + d[-3]*d[-1]*d[-1]*v,
 -2*d[-6]*v - 3/2*d[-5]*d[-1]*v + 1/3*d[-4]*d[-1]*d[-1]*v + d[-3]*d[-3]*v,
 -9/2*d[-8]*v - 9/2*d[-7]*d[-1]*v - d[-6]*d[-1]*d[-1]*v + 3/2*d[-5]*d[-3]*v + d[-4]*d[-4]*v + d[-4]*d[-3]*d[-1]*v,
 -27/4*d[-9]*v - 63/8*d[-8]*d[-1]*v - 19/8*d[-7]*d[-1]*d[-1]*v + 3/4*d[-6]*d[-3]*v + 21/8*d[-5]*d[-4]*v + 9/8*d[-5]*d[-3]*d[-1]*v + d[-4]*d[-4]*d[-1]*v,
 -393/16*d[-11]*v - 117/4*d[-10]*d[-1]*v - 189/16*d[-9]*d[-1]*d[-1]*v - 39/8*d[-8]*d[-3]*v + 27/8*d[-7]*d[-4]*v + 9/8*d[-7]*d[-3]*d[-1]*v + 27/4*d[-6]*d[-5]*v + 9/2*d[-6]*d[-4]*d[-1]*v + 21/8*d[-5]*d[-5]*d[-1]*v + d[-5]*d[-4]*d[-1]*d[-1]*v,
 -4977/20*d[-15]*v - 12207/40*d[-14]*d[-1]*v - 140449/960*d[-13]*d[-1]*d[-1]*v - 19159/160*d[-12]*d[-3]*v - 3973/160*d[-11]*d[-4]*v - 3681/160*d[-11]*d[-3]*d[-1]*v + 567/80*d[-10]*d[-5]*v - 1/12*d[-10]*d[-4]*d[-1]*v + 131/5*d[-9]*d[-6]*v + 369/20*d[-9]*d[-5]*d[-1]*v + 81/20*d[-9]*d[-4]*d[-1]*d[-1]

In [3]:
ordered_partitions(15)

[[2, 2, 2, 2, 2, 2, 2, 1],
 [2, 2, 2, 2, 2, 2, 1, 1, 1],
 [2, 2, 2, 2, 2, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 2, 2, 2, 2, 2, 2],
 [3, 2, 2, 2, 2, 2, 1, 1],
 [3, 2, 2, 2, 2, 1, 1, 1, 1],
 [3, 2, 2, 2, 1, 1, 1, 1, 1, 1],
 [3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 2, 2, 2, 2, 1],
 [4, 2, 2, 2, 2, 2, 1],
 [3, 3, 2, 2, 2, 1, 1, 1],
 [4, 2, 2, 2, 2, 1, 1, 1],
 [3, 3, 2, 2, 1, 1, 1, 1, 1],
 [4, 2, 2, 2, 1, 1, 1, 1, 1],
 [3, 3, 2, 1, 1, 1, 1, 1, 1, 1],
 [4, 2, 2, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [3, 3, 3, 2, 2, 2],
 [4, 3, 2, 2, 2, 2],
 [5, 2, 2, 2, 2, 2],
 [3, 3, 3, 2, 2, 1, 1],
 [4, 3, 2, 2, 2, 1, 1],
 [5, 2, 2, 2, 2, 1,

In [4]:
%time w = basis_up_to(15)

CPU times: user 2min 59s, sys: 80.1 ms, total: 2min 59s
Wall time: 2min 59s


In [5]:
w == basis_up_tov2(15)

True

In [6]:
%time pivots_up_to(15)

CPU times: user 2min 59s, sys: 10 ms, total: 2min 59s
Wall time: 2min 59s


[[],
 [],
 [[2]],
 [[2, 1], [1, 1, 1]],
 [[2, 2], [2, 1, 1], [1, 1, 1, 1]],
 [[2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1], [3, 2], [3, 1, 1]],
 [[2, 2, 2],
  [2, 2, 1, 1],
  [2, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1],
  [3, 2, 1],
  [3, 1, 1, 1],
  [3, 3],
  [4, 2]],
 [[2, 2, 2, 1],
  [2, 2, 1, 1, 1],
  [2, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2],
  [3, 2, 1, 1],
  [3, 1, 1, 1, 1],
  [3, 3, 1],
  [4, 2, 1],
  [4, 1, 1, 1],
  [5, 2]],
 [[2, 2, 2, 2],
  [2, 2, 2, 1, 1],
  [2, 2, 1, 1, 1, 1],
  [2, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2, 1],
  [3, 2, 1, 1, 1],
  [3, 1, 1, 1, 1, 1],
  [3, 3, 2],
  [4, 2, 2],
  [3, 3, 1, 1],
  [4, 2, 1, 1],
  [4, 1, 1, 1, 1],
  [4, 3, 1],
  [5, 2, 1],
  [5, 1, 1, 1],
  [6, 2]],
 [[2, 2, 2, 2, 1],
  [2, 2, 2, 1, 1, 1],
  [2, 2, 1, 1, 1, 1, 1],
  [2, 1, 1, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 1, 1, 1, 1, 1],
  [3, 2, 2, 2],
  [3, 2, 2, 1, 1],
  [3, 2, 1, 1, 1, 1],
  [3, 1, 1, 1, 1, 1, 1],
  [3, 3, 2, 1],
  [4, 2, 2, 1],
  [3, 3, 1, 1, 1],
  

In [7]:
series_t_q(basis_up_tov2(20))

(3*t^8 + 9*t^7 + 19*t^6 + 7*t^5 + 9*t^4 + t^3 + t^2)*q^20 + (t^8 + 7*t^7 + 16*t^6 + 7*t^5 + 8*t^4 + t^3 + t^2)*q^19 + (5*t^7 + 14*t^6 + 6*t^5 + 8*t^4 + t^3 + t^2)*q^18 + (3*t^7 + 11*t^6 + 6*t^5 + 7*t^4 + t^3 + t^2)*q^17 + (2*t^7 + 9*t^6 + 5*t^5 + 7*t^4 + t^3 + t^2)*q^16 + (7*t^6 + 5*t^5 + 6*t^4 + t^3 + t^2)*q^15 + (5*t^6 + 4*t^5 + 6*t^4 + t^3 + t^2)*q^14 + (3*t^6 + 4*t^5 + 5*t^4 + t^3 + t^2)*q^13 + (2*t^6 + 3*t^5 + 5*t^4 + t^3 + t^2)*q^12 + (3*t^5 + 4*t^4 + t^3 + t^2)*q^11 + (2*t^5 + 4*t^4 + t^3 + t^2)*q^10 + (t^5 + 3*t^4 + t^3 + t^2)*q^9 + (3*t^4 + t^3 + t^2)*q^8 + (2*t^4 + t^3 + t^2)*q^7 + (t^4 + t^3 + t^2)*q^6 + (t^3 + t^2)*q^5 + (t^3 + t^2)*q^4 + q^3*t^2 + q^2*t^2 + q*t + 1

In [8]:
series_t_q(list_of_list_of_partition_ends(basis_up_tov2(20), [5,3], True))

(3*t^6 + t^4)*q^20 + (3*t^6 + t^4)*q^19 + (2*t^6 + t^4)*q^18 + (2*t^6 + t^4)*q^17 + q^14*t^4 + (t^6 + t^4)*q^16 + q^13*t^4 + (t^6 + t^4)*q^15 + q^12*t^4 + q^11*t^4 + q^10*t^4 + q^9*t^4 + q^3*t^2